# Variational Autoencoder MNIST TensorFlow

In [ ]:
import tensorflow as tf
from keras.datasets import mnist
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
import numpy as np

### Load mnist image data

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Concatenate x_train and x_test
images = np.concatenate((x_train, x_test))

# Flatten images
images = images.reshape(images.shape[0], 784).astype('float32')

# Normalize images
images /= 255.

### Parameters for training and network architecture

In [ ]:
# hyperparameters for training
learning_rate = 0.001
epochs = 30000
batch_size = 32

# network parameters
image_dimension = 784 #mnist images are 28*28=784
layer_size = 512
latent_variable_dimension = 2

### Xavier initialization
Weight values have to be initialized. This is done randomly.
We need to shrink the variance of the weights which will in turn shrink the variance of the weighted sum.
We want the weights to have variance 1/n -> multiply each weight with sqrt(1/n) (sqrt(2/n) if using relu)

In [ ]:
def xavier(input_shape):
    """
    Initializes weights following xavier initialization
    :param input_shape: list, shape of neural network input
    :return: val: matrix, initialized weights
    """
    
    val = tf.random_normal(shape=input_shape,
                           stddev=1./tf.sqrt(input_shape[0]/2.))
    
    return val

#tf.keras.initializers.glorot_uniform

### Model weights and biases
The neural network has five sets of weights and five biases.

In [ ]:
weights = {'encoder': tf.Variable(xavier([image_dimension, layer_size])),
           'mean': tf.Variable(xavier([layer_size, latent_variable_dimension])),
           'std': tf.Variable(xavier([layer_size, latent_variable_dimension])),
           'decoder_hidden': tf.Variable(xavier([latent_variable_dimension, layer_size])),
           'decoder': tf.Variable(xavier([layer_size, image_dimension]))}

biases = {'encoder': tf.Variable(xavier([layer_size])),
          'mean': tf.Variable(xavier([latent_variable_dimension])),
          'std': tf.Variable(xavier([latent_variable_dimension])),
          'decoder_hidden': tf.Variable(xavier([layer_size])),
          'decoder': tf.Variable(xavier([image_dimension]))}

### Encoding
In the encoding layer, the image is multiplied with the weights, the bias is added, and this is put through a tanh (hyperbolic tangent) activation function.

In [ ]:
# Placeholder for image data
image_X = tf.placeholder(tf.float32, shape=[None, image_dimension])

# Define the computation in the encoder layer
Encoder_layer = tf.add(tf.matmul(image_X, weights['encoder']), biases['encoder'])
Encoder_layer = tf.nn.tanh(Encoder_layer)

# Define the computation in the hidden mean layer
Mean_layer = tf.add(tf.matmul(Encoder_layer, weights['mean']), biases['mean'])

# Define the computation in the hidden std layer
Std_layer = tf.add(tf.matmul(Encoder_layer, weights['std']), biases['std'])

### Latent vector Z and the reparametrization trick

In [ ]:
# Generate epsilon from a normal distribution with mean=0 and std=1
epsilon = tf.random_normal(shape=tf.shape(Std_layer), dtype=tf.float32, mean=0.0, stddev=1.0)

latent_vector = Mean_layer + tf.exp(0.5*Std_layer) * epsilon

### Decoding

In [ ]:
Decoder_hidden_layer = tf.add(tf.matmul(latent_vector, weights['decoder_hidden']), biases['decoder_hidden'])
Decoder_hidden_layer = tf.nn.tanh(Decoder_hidden_layer)

Decoder_output_layer = tf.add(tf.matmul(Decoder_hidden_layer, weights['decoder']), biases['decoder'])
Decoder_output_layer = tf.nn.sigmoid(Decoder_output_layer)

### Loss function and optimizer

In [ ]:
def loss_function(original_image, reconstructed_image):
    """
    The loss function for the VAE. Sum of the reconstruction loss
    and the KL divergence loss.
    :param original_image:
    :param reconstructed_image:
    :return: loss: float, the combined loss
    """
    
    # Reconstruction loss, cross-entropy loss
    rec_loss = original_image * tf.log(1e-10 + reconstructed_image) + (1-original_image) * tf.log(1e-10 + 1-reconstructed_image)
    rec_loss = -tf.reduce_sum(rec_loss, 1)
    print('reconstruction loss:', rec_loss)
    
    # KL divergence loss
    KL_div_loss = 1 + Std_layer - tf.square(Mean_layer) - tf.exp(Std_layer)
    KL_div_loss = -0.5 * tf.reduce_sum(KL_div_loss, 1)
    print('KL divergence loss:', KL_div_loss)
    
    # Sum of the two losses
    alpha = 1
    beta = 1
    network_loss = tf.reduce_mean(alpha * rec_loss + beta * KL_div_loss)
    
    return network_loss

In [ ]:
loss_value = loss_function(image_X, Decoder_output_layer)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_value)

### Execute computational graph

In [ ]:
# Initialize variables
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()
sess.run(init)

for i in range(epochs):
    index = np.random.choice(images.shape[0], 32, replace=True)
    x_batch = images[index]
    _, loss = sess.run([optimizer, loss_value], feed_dict = {image_X: x_batch})
    if i%100 == 0:
        print('Loss: {} at iteration: {}'.format(loss, i))